In [1]:
###########################################################################################################
#Preprocessamento
#Tiago Tambonis
#02/02/19
###########################################################################################################

In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_selection import VarianceThreshold

In [3]:
#Carregar 

with open('DadosTreinoCru', 'rb') as fp:
        DadosTreinoCru = pickle.load(fp)

with open('DadosTesteCru', 'rb') as fp:
        DadosTesteCru = pickle.load(fp)

In [4]:
print("Shape inicial: ", DadosTreinoCru.shape)

('Shape inicial: ', (14103, 401))


In [5]:
#Removing constant features

constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(DadosTreinoCru)

constant_columns = [column for column in DadosTreinoCru.columns if column not in 
                    DadosTreinoCru.columns[constant_filter.get_support()]]

DadosTreinoCru.drop(labels=constant_columns, axis=1, inplace=True)
DadosTesteCru.drop(labels=constant_columns, axis=1, inplace=True)

print(DadosTreinoCru.shape)
print(DadosTesteCru.shape)

(14103, 401)
(1568, 401)


In [6]:
#Removing quasi-constant features

qconstant_filter = VarianceThreshold(threshold=0.01)
qconstant_filter.fit(DadosTreinoCru)

qconstat_columns = [column for column in DadosTreinoCru.columns if column not in 
                    DadosTreinoCru.columns[qconstant_filter.get_support()]]

DadosTreinoCru.drop(labels=qconstat_columns, axis=1, inplace=True)
DadosTesteCru.drop(labels=qconstat_columns, axis=1, inplace=True)

print(DadosTreinoCru.shape)
print(DadosTesteCru.shape)

(14103, 401)
(1568, 401)


In [7]:
#Removing duplicate features

DadosTreinoCru_T = DadosTreinoCru.T
DadosTreinoCru_T.shape

DadosTreinoCru = DadosTreinoCru_T.drop_duplicates(keep='first').T

print(DadosTreinoCru.shape)
print(DadosTesteCru.shape)

(14103, 401)
(1568, 401)


In [8]:
#Removing correlated features

correlated_features = set()
correlation_matrix = DadosTreinoCru.corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i,j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

DadosTreinoCru.drop(labels=correlated_features, axis=1, inplace=True)
DadosTesteCru.drop(labels=correlated_features, axis=1, inplace=True)

print(DadosTreinoCru.shape)
print(DadosTesteCru.shape)

(14103, 401)
(1568, 401)


# Save

In [9]:
#Saving 

with open("DadosTreinoCru", "wb") as fp:   #Pickling
    pickle.dump(DadosTreinoCru, fp)   
with open("DadosTesteCru", "wb") as fp:   #Pickling
    pickle.dump(DadosTesteCru, fp) 

In [10]:
#Salvar no formato adequado am mRMR.
if True: 
    
    #Manipular as features para adequar ao mRMR.
    featuresmrmr = []
    featuresmrmr.append(DadosTreinoCru.shape[1]-1)
    for i in range(DadosTreinoCru.shape[1]-1):
        featuresmrmr.append(i)

    featuresmrmr = DadosTreinoCru.columns.values[featuresmrmr]
    
    DadosTreinomrmrCSV = DadosTreinoCru[featuresmrmr]

    normalizar = False

    if normalizar: #Normalizar?
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        y_treino_cru = np.array(DadosTreinoCru['Classe'])
        X_treino_cru = np.array(DadosTreinoCru.drop(['Classe'], 1))
        DadosTreinomrmrCSV = pd.DataFrame(np.column_stack((y_treino_cru, 
                             scaler.fit_transform(X_treino_cru))), 
                             columns=DadosTreinomrmrCSV.columns)

    #Salvar no formato adequado ao mrmr. 
    #data_converted_csv = data_converted_csv.sample(frac=1.0) #Random data
    DadosTreinomrmrCSV.to_csv("DadosTreinoCruCSVmrmr.csv", header=True, index=False)
    
    DadosTreinomrmrCSV.head()

In [11]:
print("Shapes finais \n")
print(DadosTreinoCru.shape)
print(DadosTesteCru.shape)

Shapes finais 

(14103, 401)
(1568, 401)


In [12]:
print("OK")

OK


In [13]:
DadosTesteCru.head()

,GW,GV,GT,GS,GR,GQ,GP,GY,GG,GF,...,AQ,AP,AS,AR,AT,AW,AV,AY,VK,Classe
0,0.0,0.00,0.0,0.00,0.0,5.26,0.00,0.0,5.26,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,-1.0
1,0.0,5.26,0.0,0.00,0.0,0.00,0.00,0.0,5.26,0.0,...,0.0,0.00,0.0,5.26,0.0,0.0,0.0,0.0,0.00,-1.0
2,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,-1.0
3,0.0,0.00,0.0,5.26,0.0,0.00,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,5.26,-1.0
4,0.0,0.00,0.0,0.00,0.0,0.00,5.26,0.0,0.00,0.0,...,0.0,5.26,0.0,0.00,0.0,0.0,0.0,0.0,0.00,1.0
